# Cloud-Filtered Mosaics and Shapefile Region Selection

This notebook can be used to create Landsat cloud-filtered mosaics for any time period and location. Selecting regions with shapefiles is also demonstrated. The mosaics can be output as GeoTIFF products for analysis in external GIS tools. 

The following mosaics are possible:
* Median = midpoint of spectral data
* Geomedian = Australian median product with improved spectral consistency
* Most-Recent = most recent clear pixel
* Least-Recent = least recent clear pixel
* Max-NDVI = maximum vegetation response
* Min-NDVI = minimum vegetation response

Users should review the "Cloud_Statistics" notebook for more information about the cloud statistics for any given temporal and spatial combination. An understanding of the underlying data is important for creating a valid mosaic for further analyses.

## Import Dependencies and Connect to the Data Cube

In [1]:
# Supress Warning 
import warnings
warnings.filterwarnings('ignore')

In [24]:
import os

# Load Data Cube Configuration
import datacube
dc = datacube.Datacube(app = 'my_app', config = '/home/localuser/.datacube.conf')

# Import Data Cube API
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')

## Specify Platform, Product, and Mosaic Parameters

In [3]:
# CHANGE HERE >>>>>>>>>>>>>>>>>

# Select a Product and Platform
# Examples: ghana, kenya, tanzania, sierra_leone, senegal

# product = "ls7_ledaps_ghana"
# platform = "LANDSAT_7"

product = "ls8_lasrc_ghana"
platform = "LANDSAT_8"

# Specify Mosaic Parameters
# The mosaic method to use when creating the final composite.
# One of ['median', 'geomedian', 'most_recent', 'least_recent', 'max_ndvi', 'min_ndvi']
# The options 'max' and 'min' require a spectral index to be calculated.
mosaic_method = 'median'

## Get the Extents of the Cube

In [4]:
# Print extents of the Data Cube
extents = api.get_full_dataset_extent(platform = platform, product = product, measurements=[])

latitude_extents = (min(extents['latitude'].values),max(extents['latitude'].values))
longitude_extents = (min(extents['longitude'].values),max(extents['longitude'].values))
time_extents = (min(extents['time'].values),max(extents['time'].values))

print("Latitude Extents:", latitude_extents)
print("Longitude Extents:", longitude_extents)
print("Time Extents:", time_extents)

Latitude Extents: (3.7730589405966173, 11.318637832619382)
Longitude Extents: (-3.772789446011382, 1.8863273493593828)
Time Extents: (numpy.datetime64('2013-04-13T10:10:55.000000000'), numpy.datetime64('2018-01-01T10:33:59.000000000'))


In [5]:
# The code below renders a map that can be used to view the region.
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude_extents,longitude_extents)

## Get the Regions Bounded by the Shapefiles

In [8]:
import fiona
import rasterio
import rasterio.mask
from shapely.geometry import shape

input_shp_root_pth = 'input/shapefiles/Ghana/biggest_smallest_district/'
big_distr_shp_pth = input_shp_root_pth + 'biggest_district.shp'
sml_distr_shp_pth = input_shp_root_pth + 'smallest_district.shp'

### Showing First Shapefile Region ('biggest_district.shp')

In [9]:
with fiona.open(big_distr_shp_pth, 'r') as src:
    # create a shapely geometry
    geometry = shape(src[0]['geometry'])

    # get the bounding box of the shapefile geometry
    latitude, longitude = [[None]*2, [None]*2]
    longitude[0], latitude[0] = geometry.bounds[0:2]
    longitude[1], latitude[1] = geometry.bounds[2:4]
display_map(latitude,longitude)

### Showing Second Shapefile Region ('smallest_district.shp')

In [10]:
with fiona.open(sml_distr_shp_pth, 'r') as src:
    # create a shapely geometry
    geometry = shape(src[0]['geometry'])

    # get the bounding box of the shapefile geometry
    latitude, longitude = [[None]*2, [None]*2]
    longitude[0], latitude[0] = geometry.bounds[0:2]
    longitude[1], latitude[1] = geometry.bounds[2:4]
display_map(latitude,longitude)

## Specify the Shapefile Region and Time Range to Load

In [11]:
# Path to shapefile to load data for - one of [big_distr_shp_pth, sml_distr_shp_pth].
# Note that `big_distr_shp_pth` is very resource intensive to process because it is a large area.
shp_file_pth_to_load = sml_distr_shp_pth

# Time period
time_extents = ('2017-01-01', '2018-01-01')

## Load the Data and Create the Composite

In [20]:
import numpy as np

with fiona.open(shp_file_pth_to_load, 'r') as src:
    # create a shapely geometry
    # this is done for the convenience for the .bounds property only
    shp_geom = shape(src[0]['geometry'])

    # get the bounding box of the shapefile geometry
    latitude, longitude = [[None]*2, [None]*2]
    longitude[0], latitude[0] = shp_geom.bounds[0:2]
    longitude[1], latitude[1] = shp_geom.bounds[2:4]
    
    # load data for the bounding box of the shapefile geometry
    measurements = ['red', 'green', 'blue', 'pixel_qa']
    if mosaic_method in ['max_ndvi', 'min_ndvi']:
        measurements += ['nir']
    dataset = dc.load(latitude = latitude,
                      longitude = longitude,
                      platform = platform,
                      time = time_extents,
                      product = product,
                      measurements = measurements) 

    # mask out clouds and invalid data
    from utils.data_cube_utilities.clean_mask import landsat_qa_clean_mask, landsat_clean_mask_invalid
    cloud_mask = landsat_qa_clean_mask(dataset, platform=platform)
    invalid_mask = landsat_clean_mask_invalid(dataset)
    cleaned_dataset = dataset.where(cloud_mask & invalid_mask)

    # rasterize the shapefile geometry to a boolean mask
    from datacube.utils import geometry
    src_crs = src.crs_wkt if src.crs_wkt != '' else "EPSG:4326"
    crs = geometry.CRS(src_crs)
    
    first_geometry = src[0]['geometry']
    geom = geometry.Geometry(first_geometry, crs=crs)
    geobox = dataset.geobox
    shp_mask = rasterio.features.geometry_mask(
        [geom.to_crs(geobox.crs)],
        out_shape=geobox.shape,
        transform=geobox.affine,
        all_touched=True,
        invert=True)
    
# Create the final composite.
final_dataset = cleaned_dataset.where(shp_mask)
if mosaic_method == 'median':
    from utils.data_cube_utilities.dc_mosaic import create_median_mosaic
    composite = create_median_mosaic(cleaned_dataset, cloud_mask)
elif mosaic_method == 'geomedian':
    from utils.data_cube_utilities.dc_mosaic import create_hdmedians_multiple_band_mosaic
    composite = create_hdmedians_multiple_band_mosaic(cleaned_dataset, cloud_mask)
elif mosaic_method == 'most_recent':
    from utils.data_cube_utilities.dc_mosaic import create_mosaic
    composite = create_mosaic(cleaned_dataset, cloud_mask.values)
elif mosaic_method == 'least_recent':
    from utils.data_cube_utilities.dc_mosaic import create_mosaic
    composite = create_mosaic(cleaned_dataset, cloud_mask.values, reverse_time=True)
elif mosaic_method == 'max_ndvi':
    from utils.data_cube_utilities.dc_mosaic import create_max_ndvi_mosaic
    composite = create_max_ndvi_mosaic(cleaned_dataset, cloud_mask.values)
elif mosaic_method == 'min_ndvi':
    from utils.data_cube_utilities.dc_mosaic import create_min_ndvi_mosaic
    composite = create_min_ndvi_mosaic(cleaned_dataset, cloud_mask.values)
    
final_composite = composite.where(shp_mask)

## Visualize the Composite

In [21]:
from utils.data_cube_utilities.dc_rgb import rgb
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(10,10))
rgb(final_composite, fig=fig)
plt.show()

<Figure size 1000x1000 with 1 Axes>

## Export to GeoTIFF

In [27]:
from utils.data_cube_utilities.import_export import export_slice_to_geotiff
size_str = 'small_dist' if shp_file_pth_to_load == sml_distr_shp_pth else 'big_dist'
output_dir = 'output/geotiffs/DCAL_Custom_Mosaics_and_Shapefiles'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
export_slice_to_geotiff(composite, output_dir + '/{}_{}_composite.tif'.format(size_str, mosaic_method))

In [28]:
!ls -lah output/geotiffs/DCAL_Custom_Mosaics_and_Shapefiles

total 6.2M
drwxrwxr-x 2 localuser localuser 4.0K Mar 15 14:25 .
drwxrwxr-x 3 localuser localuser 4.0K Mar 15 14:25 ..
-rw-rw-r-- 1 localuser localuser 6.2M Mar 15 14:25 small_dist_median_composite.tif
